In [1]:
import numpy as np
import itertools
import pandas as pd
from lpci import LPCI, EvaluateLPCI
from panelsplit import PanelSplit
from sklearn_quantile import RandomForestQuantileRegressor
import multiprocessing as mp
import os

# Introduction

This notebook serves as a guide for implementing the LPCI algorithm for a regression forecasting model.

We will refrain from a mathematical description in this notebook and focus on the code. Please see the README for a more technical overview.

We assume that point predictions have been obtained on a calibration and test set. Our implementation then undertakes the following steps:

1) Compute non-conformity scores (residuals) on calibration set.
2) Train a Quantile Random Forest (QRF) where X = lagged residuals and y = current calibration residual.
3) Use the fitted QRF to construct prediction interval with conditional quantiles.
4) Evaluate and plot.

Let's start with what is required to instantiate the class.



# Generate some dummy data

Let's generate some dummy calibration and test predictions for the purposes of this notebook.

In [2]:
#set up the panel data

#most importantly, you must account for data update delay. This should directly correspond to your time col.
#In our case, the data is yearly, and released with a 1 year delay.
eval_delay = 1

#set the units
unit_col = 'isocode'
units = ['DEU', 'ESP', 'GBR', 'FRA', 'ITA']

#now the time
time_col = 'year'
id_vars = [unit_col, time_col]
calibration_years = np.arange(2015, 2020, dtype = int) #recall that this is end-exclusive
test_years = np.arange(2020, 2024, dtype = int) #recall that this is end-exclusive

#construct empty panels
cal_set = pd.DataFrame()
cal_set[id_vars] = [[unit, year] for unit, year in itertools.product(units, calibration_years)]
test_set = pd.DataFrame()
test_set[id_vars] = [[unit, year] for unit, year in itertools.product(units, test_years)]

#format year column as an integer
cal_set[time_col] = cal_set[time_col].astype(int)
test_set[time_col] = test_set[time_col].astype(int)
cal_set[unit_col] = cal_set[unit_col].astype('category')
test_set[unit_col] = test_set[unit_col].astype('category')

#generate random predictions (only positive values)
preds_col = 'preds'
true_col = 'true'

#absolute because we just want positive values for our dummy data
cal_set[preds_col] = np.abs(np.random.normal(0, 10_000, cal_set.shape[0]))
cal_set[true_col] = np.abs(np.random.normal(0, 10_000, cal_set.shape[0]))

test_set[preds_col] = np.abs(np.random.normal(0, 10_000, test_set.shape[0]))
test_set[true_col] = np.abs(np.random.normal(0, 10_000, test_set.shape[0]))

We have everything we need to instantiate an instance of LPCI. Feel free to play around and check out the attributes.

In [ ]:
lpci = LPCI(
    eval_delay,
    cal_set,
    test_set, 
    unit_col=unit_col, 
    time_col=time_col, 
    preds_col=preds_col, 
    true_col=true_col
    )

#for example, note that we construct a "full dataframe" by concatenating the calibration and test sets.
#This is the .df attribute of the class

display(lpci.df)

# Step 1: Compute non-conformity scores

The current implementation assumes that the non-conformity score is equivalent to the residuals

In [ ]:
lpci.nonconformity_score(lpci.df)

# Step 2: Train a Quantile Random Forest on the calibration set

## a) Generate panel of lagged residuals and group identifiers

Let's walk through each of the steps that generate our panel

In [ ]:
#first we need to generate the residuals
df = lpci.nonconformity_score(lpci.df)
print("Below you see the residuals of the model on the calibration set.")
display(df)

In [ ]:
#then we need to generate the lagged residuals
window_size = 1

df_lag = lpci.lag(df, 'residuals', np.arange(1, window_size+1))
print(f"Below you see the residuals of the model on the calibration set with lagged residuals up to {window_size} periods.")
display(df_lag)

Note the importance of the update delay parameter. For example in our dummy data, ('DEU', 2015) represents our prediction made in 2015 for the outcome in 2016. But we would not be able to evaluate this error until 2017. The eval_delay parameter ensures this is taken into account when generating lagged residuals. We can only include the residual for 2015 as a lagged_residual feature in 2017.

Note that one can also utilise exponential smoothing on the lagged residuals and/or filling of NAs.

In [ ]:
#generate the smoothed residuals
window_size = 1

df_lag_smooth = lpci.lag(df, 'residuals', np.arange(1, window_size+1), decay = 0.8, adjust = False)
print(f"Below you see the residuals of the model on the calibration set with lagged residuals up to {window_size} periods.")
display(df_lag_smooth)

In [ ]:
#generate the smoothed residuals and fill na with 0
window_size = 1

df_lag_smooth_fillna = lpci.lag(df, 'residuals', np.arange(1, window_size+1), decay = 0.8, adjust = False, fillna = 0)
print(f"Below you see the residuals of the model on the calibration set with lagged residuals up to {window_size} periods.")
display(df_lag_smooth_fillna)

Let's proceed with simple lags (i.e. no smoothing), a window size of 2 and without filling NAs for simplicity.

In [ ]:
window_size = 2

df = lpci.lag(df, 'residuals', np.arange(1, window_size+1))
print(f"Below you see the residuals of the model on the calibration set with lagged residuals up to {window_size} periods.")
display(df)

In [ ]:
#we drop observations where there are NAs in the lagged residuals
df = df.dropna(subset = [x for x in df.columns if 'lag' in x], axis = 0, how = 'any')
print(f"Below you see the residuals of the model on the calibration set with lagged residuals up to {window_size} periods after dropping rows with NAs in the lagged residuals.")
display(df)

In [ ]:
#then we add group identifiers
cat_method = {'isocode': 'one_hot_encode'}
df = lpci.cat_engineer(df, cat_method)
print(f"Below you now see the panel required to train our QRF!")
display(df)

In [ ]:
#the complete function
window_size = 2
cat_method = {'isocode': 'one_hot_encode'}

df, features, target_col = lpci.prepare_df(
    window_size,
    cat_method = cat_method,
)

display(df)

Notice the warning generated here, we will explore this further below.

## b) Tune QRF

We are ready to tune our Quantile Random Forest. But, we must decide on a set of key parameters.

First let's consider possible cross validation strategies:

1) Time series cross validation using panel split
2) Standard k-fold cross validation

In cases where you have a small calibration dataset, eval_delay >=1 and/or prefer a larger window size, using panel_split may not be feasible.

In [ ]:
#an example of where panel split is feasible

#generate the dataframe
window_size = 1
cat_method = {'isocode': 'one_hot_encode'}
df, features, target_col = lpci.prepare_df(
    window_size,
    cat_method = cat_method,
)

#we only want to tune hyperparameters for the quantile regression forest based on data from the calibration set.
lpci._dtype_check(df, lpci.time_col, np.dtype('int64')) #check that the time_col is of type int
train_df = df[df[lpci.time_col].isin(lpci.unique_cal_time)].sort_values(by = lpci.id_vars).reset_index(drop=True)
#note that the above 3 lines are handled inside LPCI.tune() method

display(train_df)

#initialize the panel split
cv_kwargs = {
    'n_splits': 2, 
    'gap': 0, 
    'test_size': 1, 
    'progress_bar': False, 
    'plot': True
    }
panel_split = PanelSplit(train_df['year'], **cv_kwargs)



Notice that above, given the range of years in our calibration set, the eval_delay and selected window size we end up with 3 unique years in our train_df -> [2017, 2018, 2019]. Perfect, we can use panelsplit and do time-series cross validation during hyperparameter tuning.

But now see the below example.

In [ ]:
#generate the dataframe
window_size = 2
cat_method = {'isocode': 'one_hot_encode'}
df, features, target_col = lpci.prepare_df(
    window_size,
    cat_method = cat_method,
)

#we only want to tune hyperparameters for the quantile regression forest based on data from the calibration set.
lpci._dtype_check(df, lpci.time_col, np.dtype('int64')) #check that the time_col is of type int
train_df = df[df[lpci.time_col].isin(lpci.unique_cal_time)].sort_values(by = lpci.id_vars).reset_index(drop=True)
#note that the above 3 lines are handled inside LPCI.tune() method

display(train_df)

#initialize the panel split
n_splits = 2
panel_split = PanelSplit(train_df['year'], n_splits=n_splits, gap=0, test_size=1, progress_bar=False, plot = True)

With a window size of 2, our train_df is restricted to only [2018, 2019]. This is an insufficient sample to undertake time-series cross-validation.

So there is a trade-off here. In this case, we will proceed with standard cross-validation as we prefer a longer window size (i.e. more lagged residuals = more features). This will be specific to your use case.

In [ ]:
window_size = 2
cat_method = {'isocode': 'one_hot_encode'}
df, features, target_col = lpci.prepare_df(
    window_size,
    cat_method = cat_method,
)

#we only want to tune hyperparameters for the quantile regression forest based on data from the calibration set.
lpci._dtype_check(df, lpci.time_col, np.dtype('int64')) #check that the time_col is of type int
train_df = df[df[lpci.time_col].isin(lpci.unique_cal_time)].sort_values(by = lpci.id_vars).reset_index(drop=True)
#note that the above 3 lines are handled inside LPCI.tune() method

display(train_df)
cv = 3

#note that if you want to use panel_split for cross-validation, 
# then you will need a dictionary like the below to pass to the tune method
# cv_kwargs = {
#     'n_splits': 2, 
#     'gap': 0, 
#     'test_size': 1, 
#     'progress_bar': False, 
#     'plot': True
#     }


Now for the remaining parameters required for hyperparameter tuning.

In [ ]:
#quantiles

alpha = 0.1 #our significance level for the prediction interval
n_quantiles = 2 #the number of quantiles we want either side of the prediction interval

#note that alpha and n_quantiles are used in combination to generate the quantiles used for prediction in the Quantile Random Forest.
quantiles = lpci.gen_quantiles(alpha, n_quantiles)
print("These are the quantiles for which the Quantile Random Forest will generate predictions.")
display(quantiles)
del quantiles
#note that the generation of quantiles is handled inside the LPCI.tune() method

In [17]:
#grid search - you can use GridSearchCV or RandomizedSearchCV

# grid_search_method = 'GridSearchCV'
# grid_search_kwargs = {
#     'param_grid' : {
#         'n_estimators': [100, 200, 300],
#         'max_depth': [5, 10, 15],
#         'min_samples_split': [2, 5, 10],
#         'min_samples_leaf': [1, 2, 4],
#         'max_features': ['sqrt', 'log2'],
#     },
#     'scoring' : None, #careful. None uses sklearn's default scoring metric for the model (regression --> r2_score). 
#     #If you want to use something else, it will need to handle multi-output regression (since we are predicting quantiles).
#     'n_jobs' : -1,
# }

from scipy.stats import randint
grid_search_method = 'RandomizedSearchCV'
grid_search_kwargs = {
    'param_distributions' : {
        'max_depth': randint(2, 10), 
        'n_estimators': randint(10, 200),
        'min_samples_leaf': randint(2, 15),
    },
    'scoring' : None, #careful. None uses sklearn's default scoring metric for the model (regression --> r2_score). 
    #If you want to use something else, it will need to handle multi-output regression (since we are predicting quantiles).
    'n_jobs' : -1,
    'n_iter': 3
}

And now we can tune our model. Recall that our objective is to predict residuals in time t using lagged residuals as features.

In [ ]:
print(f"Target column: {target_col}")
print(f"Features: {features}")

print("Fitting the Quantile Random Forest.")
best_params, quantiles  = lpci.tune(
    df = df,
    features = features,
    target_col = target_col,
    alpha = alpha,
    n_quantiles = n_quantiles,
    grid_search_kwargs=grid_search_kwargs,
    grid_search_method=grid_search_method,
    cv_kwargs=cv,
    return_best_estimator = False   
)

best_params.update({'random_state': 42})

# Step 3: Generate prediction intervals

Now we are at crunch time! We are ready to generate prediction intervals. The entire procedure is contained in LPCI.fit_predict(), but let's break it down step by step.

In [ ]:
#1) We generate our quantiles - these should be exactly the same as the ones used in the tuning process.
lpci.gen_quantiles(alpha, n_quantiles)

In [ ]:
#2) initialize our panel split object. This is very important. 
#In contrast to the tuning process, we definitely want to utilize the panel split object here.
#This is because we want to maximize the number of observations for fitting the model.
#First, we run a check to guarantee that at least one unique time period from the calibration set is present in the df.
#This is necessary because otherwise, the first test set observation will have no observations for fitting.
# lpci._train_size_check(df)

#initialize our panelsplit object
panel_split_kwargs = {
    'gap': 0,
    'test_size': 1,
    'progress_bar': False,
    'plot': True
}
panel_split = PanelSplit(df[lpci.time_col], n_splits= lpci._get_n_splits(df[lpci.time_col].unique(), min(lpci.unique_test_time)), **panel_split_kwargs)

#empty dataframe to store the predictions
interval_df = panel_split.gen_test_labels(df[id_vars + [target_col]])

#merge back on the original predictions and target from the test set
interval_df = interval_df.reset_index() #to preserve the index
interval_df = interval_df.merge(lpci.test_preds[lpci.id_vars + [lpci.preds_col, lpci.true_col]], on = lpci.id_vars, how = 'left')
interval_df = interval_df.set_index('index')
interval_df.index.name = None

In [21]:
#3) initialize the model
best_params = {'n_estimators': 100}
quantiles = lpci.gen_quantiles(alpha, n_quantiles)
qrf = RandomForestQuantileRegressor(q = quantiles, **best_params)

#and obtain fitted estimators for each split
fitted_estimators = panel_split.cross_val_fit(
    estimator = qrf,
    X = df[features],
    y = df[target_col],
)

In [ ]:
#4) generate the predictions of the residuals. test_preds is a list of tuples [(np.array, pd.Index)]
n_jobs = -1

if n_jobs <= -1:
    num_processes = os.cpu_count()
elif n_jobs > 0:
    num_processes = n_jobs  # Use the specified number of jobs
else:
    raise ValueError(f"Invalid n_jobs value: {n_jobs}")

# Use multiprocessing for parallel processing
args = [
    (fitted_estimators[i], df.loc[test_indices, features].copy(), i)
    for i, (_, test_indices) in enumerate(panel_split.split())
]
with mp.Pool(processes=num_processes) as pool:
    test_preds = pool.map(LPCI.predict_split_mp, args)


Let's take a quick look at one of the prediction arrays.

In [ ]:
example_interval_preds = test_preds[0]

print('the first element are the predictions of the residuals (n_samples, n_quantiles)')
display(example_interval_preds[0].shape)
display(example_interval_preds[0])

print('the second element are the indices associated with the predictions')
(example_interval_preds[1])

Now the task is to compute the prediction interval. Let's take our example interval preds and see how this works. The method is contained in LPCI.gen_conf_interval()

In [ ]:
#first we split the interval preds into the upper and lower quanitiles
preds = example_interval_preds[0] #recall that this of shape (n_samples, n_quantiles)
mid_quantile = int(len(quantiles)/2)
lower_quantile_preds = preds[:, :mid_quantile]
upper_quantile_preds = preds[:, mid_quantile:]

print("The predictions for the lower quantile")
display(lower_quantile_preds)

print("The predictions for the upper quantile")
display(upper_quantile_preds)

In [ ]:
#now we compute the width of the prediction interval
widths = upper_quantile_preds - lower_quantile_preds
display(widths)

We want to find the narrowest prediction interval. The index of which is represented by i_stars. i.e. we compute the prediction interval using the pair of quantiles with the smallest width for each observation

In [ ]:
display(np.argmin(widths, axis = 1)) 
i_stars = np.argmin(widths, axis = 1) 

Using i_stars we extract the prediction for the pair of quantiles with the smallest width for each observation

In [27]:
#get the lower and upper confidence intervals
lower_conf = lower_quantile_preds[np.arange(len(i_stars)), i_stars]
upper_conf = upper_quantile_preds[np.arange(len(i_stars)), i_stars]

#get the optimal quantiles
opt_lower_q = quantiles[:mid_quantile][i_stars]
opt_upper_q = quantiles[mid_quantile:][i_stars]

In [28]:
#unpack the predictions of the intervals of the residuals
for (preds, index) in test_preds:
    #save the predictions of the residuals for each quantile
    interval_df.loc[index, [f'q_{np.round(i, 5)}' for i in quantiles]] = preds

    #compute the lower and upper confidence intervals
    lower_conf, upper_conf, opt_lower_q, opt_upper_q = lpci.gen_conf_interval(preds, quantiles)
    interval_df.loc[index, f'{target_col}_lower_conf'] = lower_conf
    interval_df.loc[index, f'{target_col}_upper_conf'] = upper_conf
    interval_df.loc[index, 'opt_lower_q'] = opt_lower_q
    interval_df.loc[index, 'opt_upper_q'] = opt_upper_q

#add the prediction of the residuals back to the point prediction to compute our final prediction interval
interval_df['lower_conf'] = interval_df[lpci.preds_col] + interval_df[f'{target_col}_lower_conf']
interval_df['upper_conf'] = interval_df[lpci.preds_col] + interval_df[f'{target_col}_upper_conf']

In [ ]:
display(interval_df)

cols_to_keep = ['isocode', 'year', 'residuals', 'preds', 'true', 'lower_conf', 'upper_conf', 'residuals_lower_conf', 'residuals_upper_conf']
display(interval_df[cols_to_keep])

Below you seen the complete function

In [ ]:
#complete function
interval_df = lpci.fit_predict(
    df = df,
    features = features,
    target_col = target_col,
    best_params = best_params,
    alpha = alpha,
    n_quantiles = n_quantiles,
    panel_split_kwargs=panel_split_kwargs,
    n_jobs = n_jobs,
    return_fitted_estimators=False #if you want to return the fitted estimators
)

display(interval_df)

# Step 4: Evaluate and plot

In [31]:
#initialize the evaluator. 
#note that the lpci instance is necessary to retain a number of key attributes e.g. unit_col, time_col etc.
evaluator = EvaluateLPCI(
    lpci,
    alpha,
    interval_df
    )

## a) Coverage metrics

Coverage metrics for different subsets of the data can be computed. 
Recall that coverage is defined as the share of times the target variable falls within the prediction interval.

In [ ]:
print("Overall coverage of the prediction intervals.")
display(evaluator.overall_coverage())

In [ ]:
print("Coverage by year")
display(evaluator.coverage_by_time())
print("Coverage by unit")
display(evaluator.coverage_by_unit())

In [ ]:
print("Coverage by bin")
bins = [0, 1, 100, 1000, 10000, 100000, float('inf')]
labels = ['0', '1-100', '100-1000', '1000-10000', '10000-100000', '100000+']

display(evaluator.coverage_by_bin(bins, labels))

## b) Plotting functions

In [ ]:
#we can plot the prediction intervals for a given year for all units
fig = evaluator.plot_intervals_year(2022)

In [ ]:
#we can also plot the prediction intervals for a given unit for all years
fig = evaluator.plot_intervals_unit('DEU')